In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
def set_size(width=347.12354, fraction=1., subplot=[1, 1]):
    """ Set aesthetic figure dimensions to avoid scaling in latex.

    Parameters
    ----------
    width: float
            Width in pts
            Default value 347.12354 is textwidth for Springer llncs
    fraction: float
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches

    From: https://jwalton.info/Embed-Publication-Matplotlib-Latex/
    """
    # Width of figure
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio * (subplot[0] / subplot[1])

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim


def init_mpl(usetex=True):
    nice_fonts = {
            # Use LaTeX to write all text
            "text.usetex": usetex,
            "font.family": "serif",
            # Use 10pt font in plots, to match 10pt font in document
            "axes.labelsize": 10,
            "font.size": 10,
            # Make the legend/label fonts a little smaller
            "legend.fontsize": 8,
            "xtick.labelsize": 8,
            "ytick.labelsize": 8,
    }

    mpl.rcParams.update(nice_fonts)
    
init_mpl()

In [ ]:
results = pickle.load(open('../hpstudy_alpha_ultimate_perf_dict.pickle', 'rb'), encoding='latin1')

In [ ]:
results_binned = pickle.load(open('../hpstudy_alpha_ultimate_perf_dict_binned.pickle', 'rb'), encoding='latin1')
print(results_binned)

In [ ]:
metric = 'rmse'
# metric = 'mae'
fraction = .75
models = [a/10 for a in list(range(0,42,2))]
nice_metric_name = {
    'mae': 'MAE',
    'rmse': 'RMSE'
}

br_a_metrics = {}
bin_ranks = results_binned[metric].keys()
for bin_rank in bin_ranks:
    if bin_rank not in br_a_metrics.keys():
        br_a_metrics[bin_rank] = {}
    for m in models:
        if m not in br_a_metrics[bin_rank].keys():
            br_a_metrics[bin_rank][m] = []
        br_a_metrics[bin_rank][m].append(results_binned[metric][bin_rank][(m, m)][0])

fig, ax = plt.subplots(figsize=set_size(fraction=fraction))
# x-axis = alpha, y-axis = RMSE, line = bin_rank
xs = list(br_a_metrics[list(br_a_metrics.keys())[0]].keys())
linestyles = ['-', '--', ':']

for bin_rank in sorted(br_a_metrics.keys()):
    ys = [100*(np.mean(br_a_metrics[bin_rank][x]) - np.mean(br_a_metrics[bin_rank][0.0]))/np.mean(br_a_metrics[bin_rank][0.0]) for x in xs]
    ax.plot(xs, ys, label=f'Bin Rank {int(bin_rank)+1}', color=str(1/(bin_rank+1.5)), linestyle=linestyles[bin_rank % len(linestyles)])

ax.set_xticks(xs[::2])
ax.legend(ncol=1, fontsize='xx-small')
ax.set_xlabel('$\\alpha$')
ax.set_ylabel('$\\Delta$' + nice_metric_name[metric] + ' (\%)')
plt.show()
fig.savefig('deepsd_binned_'+metric+'.pdf', format='pdf', bbox_inches='tight')

In [ ]:
for a in range(0, 42, 2):
    for br in range(0, 5):
        print(a/10, br, results_binned['mae'][br][(a/10, 0.0)])
    print()